**Task 2**

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset

Check Cuda Availability

In [2]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Mount google drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define data path

In [4]:
class_priority_path = '/content/drive/My Drive/data/Priority'
class_stop_path = '/content/drive/My Drive/data/Stop'

In [5]:
priority_images = [image for image in os.listdir(class_priority_path) if image.endswith('.jpg')]
stop_images = [image for image in os.listdir(class_stop_path) if image.endswith('.jpg')]
labeled_priority_images_full_path = [(os.path.join(class_priority_path, image), 1) for image in priority_images]
labeled_stop_images_full_path = [(os.path.join(class_stop_path, image), 0) for image in stop_images]
data = labeled_priority_images_full_path + labeled_stop_images_full_path

Load Data to x and y (features and label)

In [6]:
X_array = []
y_array = []


for item in data:
  x_image = cv2.imread(item[0])
  x_image = cv2.resize(x_image, (224, 224))
  X_array.append(x_image)
  y_image = item[1]
  y_array.append(y_image)

X_array = np.array(X_array)
y_array = np.array(y_array)

Convert Data to Tensor

In [7]:
X = torch.tensor(X_array, dtype=torch.float32)
y = torch.tensor(y_array, dtype=torch.int64)

Train-Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=122)
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

Load ResNet18

In [9]:
baseModel = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Create Model

In [10]:
hidden_units = 64
output_units = 2
activation_function = nn.Sigmoid()

In [11]:
headModel = nn.AdaptiveAvgPool2d((1,1))
hidden_layer = nn.Linear(1000, hidden_units)
dropout_layer = nn.Dropout(p=0.5)
output_layer = nn.Linear(hidden_units, output_units)
model_resnet18_classification = nn.Sequential(baseModel, hidden_layer, activation_function, output_layer)
model_resnet18_classification.to(device)

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

Train Data

In [12]:
epochs = 25
batch_size = 16
loss_function = nn.CrossEntropyLoss()
num_batches_train = (len(X_train) + batch_size - 1) // batch_size
optimizer = optim.Adam(model_resnet18_classification.parameters(), lr=0.001)

for epoch in range(epochs):
  loss_value = 0

  for i in range(num_batches_train):
    batch_inputs = X_train[i * batch_size:(i + 1) * batch_size].permute(0, 3, 1, 2) # (N, C, H, W, C, N)
    batch_labels = y_train[i * batch_size:(i + 1) * batch_size]

    optimizer.zero_grad()
    outputs = model_resnet18_classification(batch_inputs)
    loss = loss_function(outputs, batch_labels)
    loss.backward()
    optimizer.step()
    loss_value += loss.item()

  epoch_loss = loss_value / num_batches_train
  print(f'epoch {epoch+1} Train Loss: {epoch_loss}')

epoch 1 Train Loss: 0.3066043320455049
epoch 2 Train Loss: 0.1603153889116488
epoch 3 Train Loss: 0.07767044360700406
epoch 4 Train Loss: 0.09256063362485484
epoch 5 Train Loss: 0.053254308669190654
epoch 6 Train Loss: 0.12297067791223526
epoch 7 Train Loss: 0.02022700776395045
epoch 8 Train Loss: 0.012473368478056631
epoch 9 Train Loss: 0.010027851468246234
epoch 10 Train Loss: 0.008345210189489942
epoch 11 Train Loss: 0.007107340559167297
epoch 12 Train Loss: 0.0061573910919067104
epoch 13 Train Loss: 0.005406070941765057
epoch 14 Train Loss: 0.0047980562391641895
epoch 15 Train Loss: 0.004296920997531791
epoch 16 Train Loss: 0.0038775803864394363
epoch 17 Train Loss: 0.0035222026176358525
epoch 18 Train Loss: 0.0032177248504012823
epoch 19 Train Loss: 0.002954372045535006
epoch 20 Train Loss: 0.0027247016122074505
epoch 21 Train Loss: 0.002522907893810617
epoch 22 Train Loss: 0.0023444444553828553
epoch 23 Train Loss: 0.002185680327544871
epoch 24 Train Loss: 0.002043683510763865
ep

Save Model

In [13]:
torch.save(model_resnet18_classification, 'model_resnet18_classification.pth')

Load Model

In [14]:
model_resnet18_classification = torch.load('model_resnet18_classification.pth')

Model Evaluation

In [16]:
model_resnet18_classification.eval().to(device)

num_batches_test = (len(X_test) + batch_size - 1) // batch_size
actuals = []
predictions = []

with torch.no_grad():
  for i in range(num_batches_test):
    batch_inputs_test = X_test[i * batch_size:(i + 1) * batch_size].permute(0, 3, 1, 2)
    batch_labels_test = y_test[i * batch_size:(i + 1) * batch_size]

    outputs = model_resnet18_classification(batch_inputs_test)
    _, predicted_labels = torch.max(outputs, 1)
    actuals.append(batch_labels_test.cpu().numpy())
    predictions.append(outputs.cpu().numpy())

actuals = np.concatenate(actuals, axis=0)
predictions = np.concatenate(predictions, axis=0)

accuracy = accuracy_score(actuals, predictions)
precision = precision_score(actuals, predictions, average='weighted')
recall = recall_score(actuals, predictions, average='weighted')
f1 = f1_score(actuals, predictions, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets